In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wandb.init(project="MLOPS_Projekt", config={
    "epochs": 5,
    "batch_size": 64,
    "learning_rate": 0.001,
    "architecture": "3-layer-ffnn"
})




# Model Def


model = Net().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()



# Evaluation


torch.save(model.state_dict(), "model.pth")
wandb.save("model.pth")

c:\Users\Curdin Richert\anaconda3\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
100%|██████████| 9.91M/9.91M [00:02<00:00, 4.51MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 156kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.61MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 1.04MB/s]


Epoch 1/5, Loss: 0.0812
Epoch 2/5, Loss: 0.3681
Epoch 3/5, Loss: 0.0471
Epoch 4/5, Loss: 0.0132
Epoch 5/5, Loss: 0.0109
Test Accuracy: 97.19%
